# SnpArray.jl

The module `SnpArrays` implements the `SnpArray` type for handling biallelic genotypes. `SnpArray` is an array of `Tuple{Bool, Bool}`. If `A` is the major allele and `a` the minor allele, the coding rule is  

| Genotype | Plink | SnpArray |  
|---|---|---|  
| AA | 11 | 00 |  
| Aa | 01 | 10 |  
| aa | 00 | 11 |  
| missing | 10 | 01 |  
`SnpArray` inverts the bits in the Plink binary representation. Roughly speaking, `true` represents a minor allele. But code `01 = (false, true)` is reserved for missing genotype.

## Initialization

 `SnpArray` can be initialized from [Plink binary files](http://pngu.mgh.harvard.edu/~purcell/plink/binary.shtml), say an example data set of the MAP4 gene on chromosome 3:

In [53]:
;ls -al chr3-map4-geno.*

-rw-r--r--  1 hzhou3  staff  215043 Jul 16  2014 chr3-map4-geno.bed
-rw-r--r--  1 hzhou3  staff   25088 Jul 16  2014 chr3-map4-geno.bim
-rw-r--r--  1 hzhou3  staff   39356 Jul 16  2014 chr3-map4-geno.fam
-rw-r--r--  1 hzhou3  staff    2321 Jul 16  2014 chr3-map4-geno.log


In [54]:
include("../src/SnpArrays.jl")
using SnpArrays
map4 = SnpArray("chr3-map4-geno")

959x896 SnpArrays.SnpArray{2}:
 (false,false)  (false,false)  (false,false)  …  (true,false)   (false,false)
 (true,false)   (false,false)  (false,false)     (true,false)   (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (true,false)   (false,false)  (false,false)     (true,false)   (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)  …  (false,false)  (false,false)
 (true,false)   (false,false)  (false,false)     (true,true)    (false,false)
 (false,false)  (false,false)  (false,false)     (true,false)   (false,false)
 (false,false)  (false,false)  (true,false)      (true,false)   (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)  …  (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (true,false)   (false,false)  (f

In [55]:
size(map4)

(959,896)

Internally `SnpArray` simply stores two `BitArray`s `A1` and `A2`. For $n$ individuals and $p$ SNPs, the memory usage of `SnpArray` is $np/4$ bytes, approximately same size as the Plink `bed` file.

In [56]:
# fields of a SnpArray
fieldnames(map4)

2-element Array{Symbol,1}:
 :A1
 :A2

In [57]:
# memory usage
Base.summarysize(map4)

214896

Alternatively we can initialize `SnpArray` from a matrix of minor allele counts.

In [58]:
SnpArray(rand(0:2, 5, 3))

5x3 SnpArrays.SnpArray{2}:
 (false,false)  (true,false)  (false,false)
 (true,true)    (true,false)  (true,true)  
 (true,true)    (true,true)   (true,false) 
 (true,false)   (true,false)  (false,false)
 (true,false)   (true,true)   (true,true)  

## Summary statistics

`summarizesnps()` computes the number of minor alleles, number of missing genotypes, and minor allele frequencies (MAF) along each row and column. Calculation of MAF takes into account of missingness.

In [59]:
nmialcol, nmisscol, mafcol, nmialrow, nmissrow, mafrow = summarysnps(map4)
mafcol

896-element Array{Float64,1}:
 0.226799  
 0.00208551
 0.0260688 
 0.0135558 
 0.00104275
 0.397704  
 0.0182482 
 0.00312826
 0.00417537
 0.00834202
 0.0630865 
 0.00104275
 0.00573514
 ⋮         
 0.00104275
 0.00156413
 0.00104384
 0.00104275
 0.00521921
 0.00782065
 0.00208551
 0.00208551
 0.00208551
 0.00260688
 0.382046  
 0.00260688

In [60]:
# total number of missing genotypes
sum(nmisscol), sum(nmissrow)

(218,218)

## Subsetting

Subsetting a `SnpArray` works the same way as subsetting any other arrays.

In [61]:
# genotypes of the 1st individual
map4[1, :]

1x896 SnpArrays.SnpArray{2}:
 (false,false)  (false,false)  (false,false)  …  (true,false)  (false,false)

In [62]:
# genotypes of the 5th SNP
map4[:, 5]

959-element SnpArrays.SnpArray{1}:
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 ⋮            
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)
 (false,false)

In [63]:
# subsetting both individuals and SNPs
map4[1:5, 5:10]

5x6 SnpArrays.SnpArray{2}:
 (false,false)  (true,false)   (false,false)  …  (false,false)  (false,false)
 (false,false)  (true,false)   (false,false)     (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (false,false)  (true,false)   (false,false)     (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)

In [64]:
# filter out rare SNPs with MAF < 0.05
map4[:, mafcol .>= 0.05]

959x150 SnpArrays.SnpArray{2}:
 (false,false)  (true,false)   (false,false)  …  (true,false)   (true,false) 
 (true,false)   (true,false)   (true,false)      (true,false)   (true,false) 
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (true,false)   (true,false)   (false,false)     (true,false)   (true,false) 
 (false,false)  (false,false)  (true,false)      (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)  …  (false,false)  (false,false)
 (true,false)   (true,true)    (false,false)     (true,true)    (true,true)  
 (false,false)  (false,false)  (false,false)     (false,false)  (true,false) 
 (false,false)  (true,true)    (false,false)     (true,true)    (true,false) 
 (false,false)  (true,false)   (false,false)     (true,false)   (false,false)
 (false,false)  (false,false)  (false,false)  …  (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (true,false)   (true,false)   (f

In [65]:
# filter out individuals with genotyping success rate < 0.999
map4[nmissrow / size(map4, 2) .< 0.001, :]

817x896 SnpArrays.SnpArray{2}:
 (false,false)  (false,false)  (false,false)  …  (true,false)   (false,false)
 (true,false)   (false,false)  (false,false)     (true,false)   (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (true,false)   (false,false)  (false,false)     (true,false)   (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)  …  (false,false)  (false,false)
 (true,false)   (false,false)  (false,false)     (true,true)    (false,false)
 (false,false)  (false,false)  (false,false)     (true,false)   (false,false)
 (false,false)  (false,false)  (true,false)      (true,false)   (false,false)
 (false,false)  (false,false)  (false,false)     (false,false)  (false,false)
 (false,false)  (false,false)  (false,false)  …  (false,false)  (false,false)
 (true,false)   (false,false)  (false,false)     (true,false)   (false,false)
 (true,true)    (false,false)  (f

`sub()` and `slice()` functions work with `SnpArray`. These functions create views of subarray without copying and provide more efficiency.

In [66]:
_, _, mafcommon = summarysnps(sub(map4, :, mafcol .>= 0.05))
mafcommon

150-element Array{Float64,1}:
 0.226799 
 0.397704 
 0.0630865
 0.206465 
 0.155892 
 0.226799 
 0.37122  
 0.38634  
 0.156054 
 0.122523 
 0.123566 
 0.0866388
 0.123045 
 ⋮        
 0.390167 
 0.390511 
 0.409424 
 0.354015 
 0.411366 
 0.366771 
 0.390511 
 0.391032 
 0.395725 
 0.354015 
 0.390511 
 0.382046 

## Assignment

It is possible to assign specific genotypes to a `SnpArray` entry.

In [67]:
map4[1, 1]

(false,false)

In [68]:
map4[1, 1] = (true, false)
map4[1, 1]

(true,false)

In [69]:
map4[1, 1] = NaN
map4[1, 1]

(false,true)

In [70]:
map4[1, 1] = 0
map4[1, 1]

(false,false)

Subsetted assignment such as `map4[:, 1] = NaN` is also valid.

## Copy, convert, and imputation

In some cases we convert whole `SnpArray` to numeric arrays for computations such as PCA and Lasso. Keep in mind the storage of resultant data can be up to 32 fold that of `SnpArray`. Below are estimates of storage for some common data types. Here MAF denotes the average minor allele frequencies.

* `SnpArray`: $0.25np$ bytes  
* `Matrix{Int8}`: $np$ bytes  
* `Matrix{Float32}`: $4np$ bytes  
* `Matrix{Float64}`: $8np$ bytes  
* `SparseMatrixCSC{Float64, Int64}`: $24 \cdot NNZ \approx 24np(2MAF(1-MAF)+MAF^2) = 24np \cdot MAF(2-MAF)$ bytes. When MAF=0.25, it is $9np$ bytes. When MAF=0.017, it is $0.8np$ bypes, 10 fold smaller than `Matrix{Float64}` type.  
* `SparseMatrixCSC{Bool, Int64}`: $2np \cdot MAF \cdot 17 = 34 np \cdot MAF$ bytes. When average MAF=0.25, it is $8.5np$ bytes. When MAF=0.0235, it is $0.8np$ bytes, 10 folder smaller than `Matrix{Float64}` type.  

To be concrete, consider 2 typical data sets:  
* COPD (GWAS): $n = 6670$ individuals, $p = 630998$ SNPs, average MAF is 0.2454.
* GAW19 (sequencing study): $n = 1943$ individuals, $p = 1711766$ SNPs, average MAF is 0.00499.  

| Data Type | COPD | GAW19 |  
|---|---|---|  
| `SnpArray` | 1.05GB | 0.83GB |  
| `Matrix{Float64}` | 33.67GB | 26.61GB |  
| `SparseMatrixCSC{Float64, Int64}` | 43.5GB | 0.795GB |  
| `SparseMatrixCSC{Bool, Int64}` | 35.1GB |0.564GB |  

Apparently for data sets with majority of rare variants, converting to sparse matrices saves memory usage and often brings computational advantages too.

In [71]:
map4f64 = convert(Matrix{Float64}, map4)
# memory usage if convert to Float64
Base.summarysize(map4f64)

6874112

In [72]:
@show mean(mafcol)
map4f64sp = convert(SparseMatrixCSC{Float64, Int64}, map4)
# memory usage if convert to sparse Float64 matrix
Base.summarysize(map4f64sp)

mean(mafcol) = 0.059234694204908254


1390144

By default `convert()` method does **not** impute missing genotypes and convert them to `NaN`.

In [73]:
# number of missing genotypes
countnz(isnan(map4)), countnz(isnan(map4f64))

(218,218)

We can enforce imputation by setting optional argument `impute = true`. Imputation is done by generating two random alleles according to the minor allele frequency. This is not an optimal strategy and users should make sure genotypes are imputed with high quality using other advanced methods.

In [74]:
map4f64impute = convert(Matrix{Float64}, map4; impute = true)
countnz(isnan(map4f64impute))

0

By default `convert()` translates genotypes according to the *additive* SNP model, which essentially is the minor allele counts (0, 1 or 2). Other SNP models are *dominant* and *recessive*.

| Genotype | `SnpArray` | `model=:additive` | `model=:dominant` | `model=:recessive` |    
|---|---|---|---|---|  
| AA | 00 | 0 | 0 | 0 |  
| Aa | 10 | 1 | 0 | 2 |
| aa | 11 | 2 | 2 | 2 |
| missing | 01 | NaN | NaN | NaN | 

In [75]:
[convert(Vector{Float64}, map4[1:10, 1]; model = :additive) convert(Vector{Float64}, map4[1:10, 1]; model = :dominant) convert(Vector{Float64}, map4[1:10, 1]; model = :recessive)]

10x3 Array{Float64,2}:
 0.0  0.0  0.0
 1.0  0.0  2.0
 0.0  0.0  0.0
 1.0  0.0  2.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 1.0  0.0  2.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

By default `convert()` does **not** center and scale genotypes. Setting optional arguments `center = true, scale = true` centers genotypes at 2MAF and scales them by $[2 \cdot MAF \cdot (1 - MAF)]^{-1/2}$. Mono-allelic SNPs (MAF=0) are not centered or scaled.

In [76]:
[convert(Vector{Float64}, map4[:, 1]) convert(Vector{Float64}, map4[:, 1]; center = true, scale = true)]

959x2 Array{Float64,2}:
 0.0  -0.76593 
 1.0   0.922637
 0.0  -0.76593 
 1.0   0.922637
 0.0  -0.76593 
 0.0  -0.76593 
 1.0   0.922637
 0.0  -0.76593 
 0.0  -0.76593 
 0.0  -0.76593 
 0.0  -0.76593 
 0.0  -0.76593 
 1.0   0.922637
 ⋮             
 0.0  -0.76593 
 0.0  -0.76593 
 0.0  -0.76593 
 0.0  -0.76593 
 1.0   0.922637
 1.0   0.922637
 1.0   0.922637
 1.0   0.922637
 1.0   0.922637
 1.0   0.922637
 0.0  -0.76593 
 0.0  -0.76593 

`copy!()` is the in-place version of `convert()`. Options such as GWAS loop over SNPs and perform statistical anlaysis for each SNP. This can be achieved by

In [77]:
g = zeros(size(map4, 1))
for j = 1:size(map4, 2)
    copy!(g, map4[:, j]; model = :additive, impute = true)
    # do statistical anlaysis
end

## Empirical kinship

## PCA

In [78]:
versioninfo()

Julia Version 0.4.5
Commit 2ac304d (2016-03-18 00:58 UTC)
Platform Info:
  System: Darwin (x86_64-apple-darwin13.4.0)
  CPU: Intel(R) Core(TM) i7-3720QM CPU @ 2.60GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.3
